In [1]:
!pip install tensorflow==1.15.0

     |████████████████████████████████| 412.3MB 23kB/s 
     |████████████████████████████████| 512kB 46.9MB/s 
     |████████████████████████████████| 51kB 5.3MB/s 
     |████████████████████████████████| 3.8MB 43.9MB/s 
  Created wheel for gast: filename=gast-0.2.2-cp36-none-any.whl size=7540 sha256=97969e1b0d710e676ad733a066a414b48949320145a3f56af048e5dd6aa77539
  Stored in directory: /root/.cache/pip/wheels/5c/2e/7e/a1d4d4fcebe6c381f378ce7743a3ced3699feb89bcfbdadadd
Successfully built gast
ERROR: tensorflow-probability 0.11.0 has requirement gast>=0.3.2, but you'll have gast 0.2.2 which is incompatible.
  Found existing installation: tensorflow-estimator 2.3.0
    Uninstalling tensorflow-estimator-2.3.0:
      Successfully uninstalled tensorflow-estimator-2.3.0
  Found existing installation: gast 0.3.3
    Uninstalling gast-0.3.3:
      Successfully uninstalled gast-0.3.3
  Found existing installation: tensorboard 2.3.0
    Uninstalling tensorboard-2.3.0:
      Successfully uninsta

In [2]:
import numpy as np
import pandas as pd
import io
import tensorflow as tf
import tensorflow.train as tft
import time
from IPython.display import clear_output
from pathlib import Path
# Authentication for Managing Data
from google.colab import drive
drive.mount('/content/drive')


Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [3]:
dataOpen = np.zeros(1)
while not np.any(dataOpen):
    try:
        with open('/content/drive/My Drive/DataRaw/Detection/Training/DataOpenFault4N_11_1.csv','r') as f:
            dataOpen = np.genfromtxt(f,delimiter = ",")
    except:
        pass
np.shape(dataOpen)

(141649, 6)

In [4]:
def process(fileName):
    # Load correct File
    with open(fileName,'r') as f:
        data = np.genfromtxt(f,delimiter = ",")
    # Seperate Relative Position & Velocity data
    info = data[0,:]
    settleIndex = 300
    faultTime = np.ceil(info[0])
    if faultTime<1:
        faultTime = 1e10
    faultSat = info[1]
    faultThruster = info[2]
    faultType = info[3]

    posvelData = data[1:,:]
    noRows = np.size(posvelData, 0)//24 - settleIndex
    noCols = 6
    dataSat1 = np.zeros([noRows, 4*noCols])
    dataSat2 = np.zeros([noRows, 4*noCols])
    dataSat3 = np.zeros([noRows, 4*noCols])
    dataSat4 = np.zeros([noRows, 4*noCols])
    dataSat5 = np.zeros([noRows, 4*noCols])
    dataSat6 = np.zeros([noRows, 4*noCols])

    faultLabel = np.zeros([noRows, 1])
   
    faultLabel[np.arange(noRows)>faultTime-settleIndex] = 1
    settleOffset = settleIndex * 24 
    dataSat1[:,0*noCols:1*noCols] = posvelData[settleOffset + 0::24,0:noCols]
    dataSat1[:,1*noCols:2*noCols] = posvelData[settleOffset + 1::24,0:noCols]
    dataSat1[:,2*noCols:3*noCols] = posvelData[settleOffset + 2::24,0:noCols]
    dataSat1[:,3*noCols:4*noCols] = posvelData[settleOffset + 3::24,0:noCols]

    dataSat2[:,0*noCols:1*noCols] = posvelData[settleOffset + 4::24,0:noCols]
    dataSat2[:,1*noCols:2*noCols] = posvelData[settleOffset + 5::24,0:noCols]
    dataSat2[:,2*noCols:3*noCols] = posvelData[settleOffset + 6::24,0:noCols]
    dataSat2[:,3*noCols:4*noCols] = posvelData[settleOffset + 7::24,0:noCols]

    dataSat3[:,0*noCols:1*noCols] = posvelData[settleOffset + 8::24,0:noCols]
    dataSat3[:,1*noCols:2*noCols] = posvelData[settleOffset + 9::24,0:noCols]
    dataSat3[:,2*noCols:3*noCols] = posvelData[settleOffset + 10::24,0:noCols]
    dataSat3[:,3*noCols:4*noCols] = posvelData[settleOffset + 11::24,0:noCols]

    dataSat4[:,0*noCols:1*noCols] = posvelData[settleOffset + 12::24,0:noCols]
    dataSat4[:,1*noCols:2*noCols] = posvelData[settleOffset + 13::24,0:noCols]
    dataSat4[:,2*noCols:3*noCols] = posvelData[settleOffset + 14::24,0:noCols]
    dataSat4[:,3*noCols:4*noCols] = posvelData[settleOffset + 15::24,0:noCols]

    dataSat5[:,0*noCols:1*noCols] = posvelData[settleOffset + 16::24,0:noCols]
    dataSat5[:,1*noCols:2*noCols] = posvelData[settleOffset + 17::24,0:noCols]
    dataSat5[:,2*noCols:3*noCols] = posvelData[settleOffset + 18::24,0:noCols]
    dataSat5[:,3*noCols:4*noCols] = posvelData[settleOffset + 19::24,0:noCols]

    dataSat6[:,0*noCols:1*noCols] = posvelData[settleOffset + 20::24,0:noCols]
    dataSat6[:,1*noCols:2*noCols] = posvelData[settleOffset + 21::24,0:noCols]
    dataSat6[:,2*noCols:3*noCols] = posvelData[settleOffset + 22::24,0:noCols]
    dataSat6[:,3*noCols:4*noCols] = posvelData[settleOffset + 23::24,0:noCols]

    sats = np.arange(6)
    sats = np.repeat(sats,noRows)
    sats = sats.reshape(-1,1)
    time = np.arange(settleIndex,noRows+settleIndex)

    time = np.tile(time,6).reshape(-1,1)

    data = np.concatenate((dataSat1,dataSat2,dataSat3,dataSat4,dataSat5,dataSat6),0)
    labels = np.concatenate((faultLabel,faultLabel,faultLabel,faultLabel,faultLabel,faultLabel),0)
    data = np.concatenate((data,labels,time,sats),axis = 1)

    return data # ds3.map(lambda a,b,c: (a,b)), ds3.map(lambda a,b,c: c)
 


In [5]:
def create_tfrecord(filePath,fileName, data):
    # Create tfrecord
    
    header = ['x1','y1','z1','vx1','vy1','vz1',
                'x2','y2','z2','vx2','vy2','vz2',
                'x3','y3','z3','vx3','vy3','vz3',
                'x4','y4','z4','vx4','vy4','vz4',
                'label','time','sat']
    # Create dict
    x1  = tft.Feature(float_list = tft.FloatList(value = data[:,0]))
    y1  = tft.Feature(float_list = tft.FloatList(value = data[:,1]))
    z1  = tft.Feature(float_list = tft.FloatList(value = data[:,2]))
    vx1 = tft.Feature(float_list = tft.FloatList(value = data[:,3]))
    vy1 = tft.Feature(float_list = tft.FloatList(value = data[:,4]))
    vz1 = tft.Feature(float_list = tft.FloatList(value = data[:,5]))

    x2  = tft.Feature(float_list = tft.FloatList(value = data[:,6]))
    y2  = tft.Feature(float_list = tft.FloatList(value = data[:,7]))
    z2  = tft.Feature(float_list = tft.FloatList(value = data[:,8]))
    vx2 = tft.Feature(float_list = tft.FloatList(value = data[:,9]))
    vy2 = tft.Feature(float_list = tft.FloatList(value = data[:,10]))
    vz2 = tft.Feature(float_list = tft.FloatList(value = data[:,11]))

    x3  = tft.Feature(float_list = tft.FloatList(value = data[:,12]))
    y3  = tft.Feature(float_list = tft.FloatList(value = data[:,13]))
    z3  = tft.Feature(float_list = tft.FloatList(value = data[:,14]))
    vx3 = tft.Feature(float_list = tft.FloatList(value = data[:,15]))
    vy3 = tft.Feature(float_list = tft.FloatList(value = data[:,16]))
    vz3 = tft.Feature(float_list = tft.FloatList(value = data[:,17]))

    x4  = tft.Feature(float_list = tft.FloatList(value = data[:,18]))
    y4  = tft.Feature(float_list = tft.FloatList(value = data[:,19]))
    z4  = tft.Feature(float_list = tft.FloatList(value = data[:,20]))
    vx4 = tft.Feature(float_list = tft.FloatList(value = data[:,21]))
    vy4 = tft.Feature(float_list = tft.FloatList(value = data[:,22]))
    vz4 = tft.Feature(float_list = tft.FloatList(value = data[:,23]))

    label = tft.Feature(int64_list = tft.Int64List(value = data[:,24].astype(int)))
    time  = tft.Feature(int64_list = tft.Int64List(value = data[:,25].astype(int)))
    sats  = tft.Feature(int64_list = tft.Int64List(value = data[:,26].astype(int)))



    feature_dict = {'x1':x1,'y1':y1,'z1':z1,'vx1':vx1,'vy1':vy1,'vz1':vz1,
                    'x2':x2,'y2':y2,'z2':z2,'vx2':vx2,'vy2':vy2,'vz2':vz2,
                    'x3':x3,'y3':y3,'z3':z3,'vx3':vx3,'vy3':vy3,'vz3':vz3,
                    'x4':x4,'y4':y4,'z4':z4,'vx4':vx4,'vy4':vy4,'vz4':vz4,
                    'label':label,'time':time,'sats':sats}
    features = tft.Features(feature = feature_dict)
    DataExample = tft.Example(features = features)

    with tf.python_io.TFRecordWriter(filePath+fileName) as writer:
        writer.write(DataExample.SerializeToString())
    return


In [6]:
# Testing
path = '/content/drive/My Drive/DataRaw/Detection/Training/'
nfPattern = path + 'DataNoFault4N_*.csv'
openPattern = path + 'DataOpenFault4N_*.csv'
closedPattern= path + 'DataClosedFault4N_*.csv'
globPatterns = [nfPattern,openPattern,closedPattern]
# Seed is 6 DO NOT CHANGE
seed = 6

# Define model parameters
print(closedPattern)
# Create Dataset
listdir = tf.io.gfile.glob(globPatterns)
print(len(listdir))
fileList = np.sort(listdir)
sortedList = np.sort(listdir)
print(sortedList[:5])
np.random.seed(seed)
np.random.shuffle(fileList)


/content/drive/My Drive/DataRaw/Detection/Training/DataClosedFault4N_*.csv
51502
['/content/drive/My Drive/DataRaw/Detection/Training/DataClosedFault4N_11_1.csv'
 '/content/drive/My Drive/DataRaw/Detection/Training/DataClosedFault4N_11_10.csv'
 '/content/drive/My Drive/DataRaw/Detection/Training/DataClosedFault4N_11_100.csv'
 '/content/drive/My Drive/DataRaw/Detection/Training/DataClosedFault4N_11_101.csv'
 '/content/drive/My Drive/DataRaw/Detection/Training/DataClosedFault4N_11_102.csv']


In [7]:
trainlist = ['/content/drive/My Drive/Data/Detection/Training/TrainCorrected_478.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_499.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_20.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_205.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_326.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_383.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_330.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_223.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_68.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_324.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_460.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_221.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_43.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_94.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_232.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_510.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_46.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_366.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_153.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_317.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_252.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_405.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_293.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_434.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_142.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_454.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_53.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_134.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_309.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_144.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_273.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_296.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_486.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_415.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_448.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_402.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_348.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_226.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_204.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_473.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_332.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_504.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_501.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_349.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_147.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_452.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_371.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_427.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_8.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_69.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_63.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_431.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_379.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_1.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_506.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_356.tfrecord',
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_151.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_227.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_26.tfrecord', 
            '/content/drive/My Drive/Data/Detection/Training/TrainCorrected_465.tfrecord']
print(len('/content/drive/My Drive/Data/Detection/Training/TrainCorrected'))
starts = []
for f in trainlist:
    print(f[63:-9])
    starts.append(int(f[63:-9]))

62
478
499
20
205
326
383
330
223
68
324
460
221
43
94
232
510
46
366
153
317
252
405
293
434
142
454
53
134
309
144
273
296
486
415
448
402
348
226
204
473
332
504
501
349
147
452
371
427
8
69
63
431
379
1
506
356
151
227
26
465


In [8]:
nFiles = len(fileList)
nPerRecord = 100
start = 0
filePathWrite = '/content/drive/My Drive/Data/Detection/Training/'
# Read start and beginning from file register
with open(filePathWrite+"FileRegister.csv",'r') as f:
    register = np.genfromtxt(f, delimiter=',').astype(int)
if not np.shape(register) == (3,):
    start = register[-1,1]
    fileIndex = register[-1,2] + 1
else:
    start = 0
    fileIndex = 0

end = start + nPerRecord
tS = time.perf_counter()
durAverage = 0
fileNameWrite = "TrainCorrected_"
counter = 0
fileListWritten = []
for start in starts:
    start = start * 100
    end = start + 100
    print("Reading from {0} to {1}".format(start,end) )
    fileNameRead = fileList[start]
    fileListWritten.append(fileNameRead)
    #data = process(fileNameRead)
    filesLeft = end < nFiles
    for fileNameRead in fileList[start+1: end if filesLeft else None]:
       #data = np.concatenate((data,process(fileNameRead)),axis = 0)
       fileListWritten.append(fileNameRead)

    # Write TFRecord file 
    #fileName = fileNameWrite + str(start//100) + ".tfrecord"
    #create_tfrecord(filePathWrite,fileName,data)
    # Uodate File Register
    #with open(filePathWrite+'FileRegister.csv','a') as f:
    #    update = str(start)+ ',' + str(end) + ','+ str(fileIndex) + '\n' 
    #    f.write(update)
    tE = time.perf_counter()
    dur = tE-tS
    if durAverage == 0 :
        durAverage = dur
    else:
        durAverage = 0.9*durAverage +0.1*dur 
    tS = tE
    fileIndex += 1
    counter +=1
    #print("Written File " + fileName)
    print("Average time per file: {}".format(durAverage))
    print("Estimated minutes left: {}".format((len(starts)-counter)*durAverage/60))



Reading from 47800 to 47900
Average time per file: 0.00039741899990985985
Estimated minutes left: 0.0003907953499113622
Reading from 49900 to 50000
Average time per file: 0.0003820551999069721
Estimated minutes left: 0.0003693200265767397
Reading from 2000 to 2100
Average time per file: 0.00035870607991910224
Estimated minutes left: 0.0003407707759231471
Reading from 20500 to 20600
Average time per file: 0.00033906297193652795
Estimated minutes left: 0.0003164587738074261
Reading from 32600 to 32700
Average time per file: 0.00031991637474516215
Estimated minutes left: 0.00029325667684973195
Reading from 38300 to 38400
Average time per file: 0.0003027091372618316
Estimated minutes left: 0.0002724382235356484
Reading from 33000 to 33100
Average time per file: 0.0002885290235354501
Estimated minutes left: 0.00025486730412298096
Reading from 22300 to 22400
Average time per file: 0.00027658882117545937
Estimated minutes left: 0.00023971031168539813
Reading from 6800 to 6900
Average time per

60
6000


Checking Fault Level

In [ ]:
filePathWrite = '/content/drive/My Drive/Data/Detection/Training/'
# Read start and beginning from file register
with open(filePathWrite+"FaultRegister.csv",'r') as f:
    register = np.genfromtxt(f, delimiter=',').astype(int)
    start = int(100 * (register[-1,0] + 1))
print(register)
print(start)
fileIndex = 0
nFiles = len(fileList)
nPerRecord = 100

end = start + nPerRecord
tS = time.perf_counter()
durAverage = 0
fileNameWrite = "Train_"
fileCounter = 0 
while end  < nFiles:
    print("Reading from {0} to {1}".format(start,end) )
    fileNameRead = fileList[start]
    data = process(fileNameRead)
    filesLeft = end < nFiles
    faultCounter = 0
    faultSeverityMean = 0
    for fileNameRead in fileList[start+1: end if filesLeft else None]:
        with open(fileNameRead,'r') as f:
            data = np.genfromtxt(f,delimiter = ",")
        info = data[0,:]
        faultSeverity = info[4]
        if faultSeverity > 0.01:
            faultCounter += 1
        faultSeverityMean = faultSeverityMean + faultSeverity 
    faultSeverityWith = faultSeverityMean / 100
    faultSeverityWO = faultSeverityMean/faultCounter


    # Uodate File Register
    with open(filePathWrite+'FaultRegister.csv','a') as f:
        update = str(start//100)+ ',' + str(faultSeverityWith) + ','+ str(faultSeverityWO) + ','+ str(faultCounter) + '\n' 
        f.write(update)
    start = end
    end = start + nPerRecord
    tE = time.perf_counter()
    dur = tE-tS
    if durAverage == 0 :
        durAverage = dur
    else:
        durAverage = 0.9*durAverage +0.1*dur 
    tS = tE

    print("Average time per file: {}".format(durAverage))
    print("Estimated minutes left: {}".format((nFiles-start)/(nPerRecord)*durAverage/60))

    #Every 10 files clear output
    if fileCounter >= 9:
        clear_output()
        fileCounter = 0
    else: 
        fileCounter +=1


[[  0   0   0  58]
 [  1   0   0  50]
 [  2   0   0  51]
 ...
 [512   0   0  53]
 [513   0   0  58]
 [514   0   0  56]]
51500


In [ ]:
faultSeverityWO

NameError: ignored